# load depenedencies, get shape parameters

In [ ]:
source('~/imports.R')
library(coda)

In [ ]:
dname = '~/eckertlab/brandon/teakettle/dependencies/'
files = c()
for (f in list.files(dname)){
    if (endsWith(f,'RDS'))
    {
        cat(sprintf ("loading %s file\n",f))
        Fname = fname(dname,f)
        obj = strsplit(f,split='.RDS')[[1]]
        assign(obj,readRDS(Fname))    
    }
}

In [ ]:
usgibbs   <- readRDS('~/teakettle/data/gibbs/first/end_of_cyc_files/usgibbs_all_cycs.RDS')
upgibbs   <- readRDS('~/teakettle/data/gibbs/first/end_of_cyc_files/upgibbs_all_cycs.RDS')
ped.gibbs <- readRDS('~/teakettle/data/gibbs/first/end_of_cyc_files/pedgibbs_all_cycs.RDS')
ncyc      <- 150000

In [ ]:
c(len(usgibbs),len(upgibbs),len(ped.gibbs))

# get plot posts so we include the full mapping area

In [ ]:
pc <- matrix(rep(0,8),ncol=2)
xmin <- min(adult.loc[,1]) - 10
ymin = min(adult.loc[,2]) - 10
xmax = max(adult.loc[,1]) + 10
ymax = max(adult.loc[,2]) + 10
pc[4,1] <- pc[1,1] <- xmax
pc[2,2] <- pc[1,2] <- ymax
pc[3,1] <- pc[2,1] <- xmin
pc[4,2] <- pc[3,2] <- ymin
pcor <- pc

outpar <- matrix(NA,2,4) 
#burn  <- 3000                              #end of burn-in - original script had burn @ 3K, paper had 30K
burn   <- 30000                              #end of burn-in
thin   <- seq(burn,(length(usgibbs)),by=1500)  #thin the gibbs sequence
gmat   <- cbind(usgibbs[thin],upgibbs[thin])
outpar[,1]       <- apply(gmat,2,mean)
outpar[,2]       <- apply(gmat,2,sd)
outpar[,3:4]     <- t(apply(gmat,2,quantile,c(.025,.975)))
colnames(outpar) <- c('estimate','se','.025','.975')
rownames(outpar) <- c('us','up')
signif(outpar,2)

In [ ]:
c(mean.func(mean(usgibbs)),mean.func(mean(usgibbs[thin])))

In [ ]:
c(mean.func(mean(upgibbs)),mean.func(mean(upgibbs[thin])))

# plot shape parameters in R

In [ ]:
par(mfcol=c(2,1))
plot(usgibbs,type='l',xaxt='n')
abline(h=mean(usgibbs[thin]))
axis(1,at=seq(5000,150000,by=5000))
plot(upgibbs,type='l',xaxt='n')
abline(h=mean(upgibbs[thin]))
axis(1,at=seq(5000,150000,by=5000))

In [ ]:
effectiveSize(usgibbs[30000:len(usgibbs)])

In [ ]:
effectiveSize(upgibbs[30000:len(upgibbs)])

In [ ]:
len(upgibbs)

## Which parent pairs came up which percentage of the time?  

In [ ]:
c(burn,ncyc,nadult,noff)

In [ ]:
par.mat <- numeric(0)
# f.vec <- m.vec   <- matrix(0,noff,nadult+1) # never used
for(k in 1:noff)
{
    print(k)
    p.m <- matrix(0,nadult+1,nadult+1)
    for(d in burn:ncyc)
    {
        p.m[ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]] <- 1 + p.m[ped.gibbs[[d]][k,1],ped.gibbs[[d]][k,2]] 
    }
    p.m <- p.m/sum(p.m)
    par.mat <- append(par.mat,list(p.m))
}

In [ ]:
# Identify most frequent parent pair for each offspring.  M is most likely mother, f is most likely father, 
## per the percentage of time they were matched with offspring.
m <- rep(NA,noff)
f <- rep(NA,noff)
per <- rep(NA,noff) 
for(k in 1:noff){
      v <- matrix(0,nadult+1,nadult+1)
      v[which(par.mat[[k]][,]==max(par.mat[[k]]))]<-1

      f[k]<- which(apply(v,1,sum)==1)
      m[k]<- which(apply(v,2,sum)==1)
      per[k] <- max(par.mat[[k]])
}

In [ ]:
table(m)

In [ ]:
table(f)

In [ ]:
# save RDS
DIR = '/home/lindb/teakettle/data/gibbs/first/end_of_cyc_files'
objects <- c('par.mat','m','f','per','outpar')

for (obj in objects){
    filename = sprintf("%s.RDS",obj)
    cat(sprintf('saving %s\n',filename))
    saveRDS(get(obj),fname(DIR,filename))
}

## Map parent-offspring relationships

In [ ]:
#cornerposts  
X <- pcor[,1]
Y <- pcor[,2]
plot(X,Y)
#adults
#d2 <- w
d2 <- rep(4,nadult)
symbols(adult.loc[,1],adult.loc[,2],squares=d2,inches=F,fg=4,add=T)
# #plots
# d3 <- rep(2,nrow(plot.loc))
# symbols(plot.loc[,1],plot.loc[,2],squares=d3,inches=F,fg=1,add=T)
#seedlings
d4 <- rep(4,noff)
symbols(off.loc[,1],off.loc[,2],circles=d4,inches=F,fg=3,add=T)

## plot parent-offspring connections

In [ ]:
#cornerposts  
X <- pcor[,1]
Y <- pcor[,2]
plot(X,Y)
#adults
#d2 <- w
d2 <- rep(4,nadult)
symbols(adult.loc[,1],adult.loc[,2],squares=d2,inches=F,fg=4,add=T)
#seedlings
d4 <- rep(4,noff)
symbols(off.loc[,1],off.loc[,2],circles=d4,inches=F,fg=3,add=T)

x0 <- numeric(0)
y0 <- x0
x1 <- x0
y1 <- x0
x0b <- x0
y0b <- x0
x1b <- x0
y1b <- x0
x0c <- x0
y0c <- x0
x1c <- x0
y1c <- x0
x0.o <- x0
y0.o <- x0
x1.o <- x0
y1.o <- x0
x0b.o <- x0
y0b.o <- x0
x1b.o <- x0
y1b.o <- x0

for (k in 1:noff){
    if(m[k]<nadult+1){      # if mother is in the plot
        # offspring locations
        x0 <- c(x0,off.loc[k,1])
        y0 <- c(y0,off.loc[k,2])
        # mother locations
        x1 <- c(x1,adult.loc[m[k],1])
        y1 <- c(y1,adult.loc[m[k],2])
        
        if(f[k]<nadult+1) { # if father is in the plot
            # mother locations
            x0b <- c(x0b,adult.loc[m[k],1])
            y0b <- c(y0b,adult.loc[m[k],2])
            # father locations
            x1b <- c(x1b,adult.loc[f[k],1])
            y1b <- c(y1b,adult.loc[f[k],2])
            x0c <- c(x0c,adult.loc[f[k],1])
            y0c <- c(y0c,adult.loc[f[k],2])
            # offspring locations
#             x1c <- c(x0,off.loc[k,1]) # this can't be right
#             y1c <- c(y0,off.loc[k,2]) # this can't be right
            x1c <- c(x1c,off.loc[k,1])  # replace with this
            y1c <- c(y1c,off.loc[k,2])  # replace with this
        } 
        if(f[k]==nadult+1){ # if father is outside the plot
            # mother locations
            x0b.o <- c(x0b.o,adult.loc[m[k],1])
            y0b.o <- c(y0b.o,adult.loc[m[k],2])
            # out-of-plot father locations
            md <- which(d.edge.a[m[k],]==min(d.edge.a[m[k],]))
            x1b.o[k] <- edgemid[md,1]
            y1b.o[k] <- edgemid[md,2] 
        }
    }
    if(m[k]==nadult+1){     # if mother is outside the plot
        x0.o <- c(x0.o,off.loc[k,1])
        y0.o <- c(y0.o,off.loc[k,2])
        md <- which(d.edge.o[k,]==min(d.edge.o[k,]))
        x1.o <- c(x1.o,edgemid[md,1])
        y1.o <- c(y1.o,edgemid[md,2])
    }
}

#segments(x0b,y0b,x1b,y1b,col="red",lwd=1) #inplot father to mother
segments(x0c,y0c,x1c,y1c,lty=2,lwd =1) #inplot father to offspring
#segments(x0,y0,x1,y1,lwd=1)  #inplot mother to offspring

In [ ]:
c(len(x1b),len(y1b),len(x1c),len(y1c))

## hist of mother-off-father distances

In [ ]:
distance = function(a0,a1,b0,b1){
    ddd <- rep(0,len(a0))
    for (i in 1:len(a0)){
        adiff = (a0[i]-a1[i])**2
        bdiff = (b0[i]-b1[i])**2
        diff = sqrt(adiff+bdiff)
        ddd[i] <- diff
    }
    return (ddd)
}

#### mother-offspring

In [ ]:
#x0 y0 x1 y1
m.off <- distance(x0,x1,y0,y1)

In [ ]:
c(mean(m.off),median(m.off))

In [ ]:
mean.func(mean(usgibbs))

In [ ]:
mean.func(mean(usgibbs[thin]))

In [ ]:
hist(m.off, main="mother-offspring distances",xlab ="meters")

#### father-offspring

In [ ]:
f.off <- distance(x0c,x1c,y0c,y1c)

In [ ]:
c(len(x0c),len(x1c),len(y0c),len(y1c))

In [ ]:
hist(f.off,main="father-offspring distances",xlab="meters")

#### mother-father

In [ ]:
c(len(x0b),len(y0b),len(x1b),len(y1b))

In [ ]:
m.f <- distance(x0b,x1b,y0b,y1b)

In [ ]:
hist(m.f,main="mother-father distances",xlab="meters")

# rep 2

In [ ]:
usgibbs2 <- readRDS('~/teakettle/data/gibbs/first_rep2/end_of_cyc_files/usgibbs_all_cycs.RDS')
upgibbs2 <- readRDS('~/teakettle/data/gibbs/first_rep2/end_of_cyc_files/upgibbs_all_cycs.RDS')
#ped.gibbs <- readRDS('~/teakettle/data/gibbs/first_rep2/end_of_cyc_files/pedgibbs_all_cycs.RDS')

In [ ]:
par(mfcol=c(2,1))
plot(usgibbs2,type='l')
abline(h=mean(usgibbs2[thin]))
plot(upgibbs2,type='l')
abline(h=mean(upgibbs2[thin]))

In [ ]:
c(mean(usgibbs[thin]),mean(usgibbs2[thin]))

In [ ]:
c(mean.func(mean(usgibbs[thin])),mean.func(mean(usgibbs2[thin])))

In [ ]:
len(usgibbs2)

In [ ]:
effectiveSize(usgibbs2[30000:len(usgibbs2)])

In [ ]:
effectiveSize(upgibbs2)

# rep 3

In [ ]:
usgibbs3 <- readRDS('~/teakettle/data/gibbs/first_rep3/end_of_cyc_files/usgibbs_all_cycs.RDS')
upgibbs3 <- readRDS('~/teakettle/data/gibbs/first_rep3/end_of_cyc_files/upgibbs_all_cycs.RDS')
#ped.gibbs <- readRDS('~/teakettle/data/gibbs/first_rep3/end_of_cyc_files/pedgibbs_all_cycs.RDS')

In [ ]:
par(mfcol=c(2,1))
plot(usgibbs3,type='l')
abline(h=mean(usgibbs3[thin]))
plot(upgibbs3,type='l')
abline(h=mean(upgibbs3[thin]))

In [ ]:
c(mean(usgibbs[thin]),mean(usgibbs2[thin]),mean(usgibbs3[thin]))

In [ ]:
c(mean.func(mean(usgibbs[thin])),mean.func(mean(usgibbs2[thin])),mean.func(mean(usgibbs3[thin])))

In [ ]:
effectiveSize(usgibbs3[30000:len(usgibbs3)])

In [ ]:
effectiveSize(upgibbs3[30000:len(upgibbs3)])

In [ ]:
effectiveSize(upgibbs3)

In [ ]:
library(adegenet)

In [ ]:
library(PopGenReport)

In [ ]:
install.packages('PopGenReport')

In [ ]:
install.packages('png')

In [ ]:
install.packages('RgoogleMaps')